# Dynamics systems and prepare data



## Voltera-Lotki equation is used to describe the dynamics of biological systems [Lotka–Volterra equations](https://en.wikipedia.org/wiki/Lotka-Volterra_equations) consists of two differential equations:

\begin{align*}
&\frac{\mathrm{d}x}{\mathrm{d}t} = ax - bxy,\\[1em]
&\frac{\mathrm{d}y}{\mathrm{d}t} = cxy - dy,\\[1em]
\end{align*}

with the following parameter values:
```python
a=1.0
b=0.3
c=0.2
d=1.0
```
and the initial value at time $t = 0$ with $x = 10$ and $y = 5$.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy.integrate import solve_ivp
from improve_dmdc import Improved_DMDc_new, Standard_DMDC
from controller import mpc_controller_lotka
import cvxpy as cp

np.random.seed(42)

In [ ]:
# Lotka-Volterra parameters
# Lotka-Volterra parameters
a, b, c, d = 1.0, 0.3, 0.2, 1.0
integrator_keywords = {'rtol': 1e-12, 'method': 'LSODA', 'atol': 1e-12}

# Volterra-Lotka ODE
def ode_Volterra_Lotka(t, xy, uu):
    x, y = xy
    dx_dt = a * x - b * x * y
    dy_dt = c * x * y - d * y + uu
    return [dx_dt, dy_dt]

# Simulation function
def SciPy_VL(t_begin, t_end, x_init, y_init, t_space, u_con=None):
    if u_con is not None:
        u_interp = scipy.interpolate.interp1d(t_space, u_con, kind='previous', bounds_error=False, fill_value=(u_con[0], u_con[-1]))
        def ode_fun(t, xy):
            return ode_Volterra_Lotka(t, xy, u_interp(t))
    else:
        def ode_fun(t, xy):
            return ode_Volterra_Lotka(t, xy, 0.0)
    num_sol = solve_ivp(ode_fun, [t_begin, t_end], [x_init, y_init], **integrator_keywords, t_eval=t_space)
    return num_sol.y[0], num_sol.y[1]

# Simulation parameters
t_init, t_end, delta_t = 0, 100, 0.003
x_init, y_init = 10, 5
N_t = int((t_end - t_init) / delta_t) + 1
t_array = np.linspace(t_init, t_end, N_t)

# # # if we use random control input
# u_con = (2 * np.sin(t_array) * np.sin(t_array / 10)) ** 2
# x_solved, y_solved = SciPy_VL(t_begin=t_init, t_end=t_end, x_init=x_init, y_init=y_init, t_space=t_array, u_con=u_con)

# if we assumed that we have known the model to design a perfect MPC
x_sim = [np.array([x_init, y_init])]
u_con_list = []
for i in range(N_t):
    # find the control action at time i
    current_control = mpc_controller_lotka(a, b, c, d, x_sim[-1], horizon = 5, dt=delta_t, nx=2, nu=1)

    # simulate the system to find next step
    sol = solve_ivp(lambda t, x: ode_Volterra_Lotka(t, x, current_control.squeeze()), [0, delta_t], x_sim[-1], t_eval=[delta_t], **integrator_keywords)
    x_next = sol.y[:, -1]
    x_sim.append(x_next)
    u_con_list.append(current_control)
x_solved = np.array([x[0] for x in x_sim[1:]])
y_solved = np.array([x[1] for x in x_sim[1:]])
u_con = np.array([u[0] for u in u_con_list])


# Training data
N_train = int(N_t * 0.8)
x_train = x_solved[:N_train]
y_train = y_solved[:N_train]
t_train = t_array[:N_train]
X_data = np.vstack((x_train, y_train))
U_control = u_con[:N_train-1].reshape(1, -1)

# # Subplot 1: State Trajectories
# plt.figure(figsize=(12, 5))
# plt.plot(t_array, x_solved, 'b-', label='Original x')
# plt.plot(t_array, y_solved, 'g-', label='Original y')
# plt.title('State Trajectories')
# plt.xlabel('Time')
# plt.ylabel('Population')
# plt.legend()
# plt.grid(True)

# plt.figure(figsize=(12, 5))
# plt.plot(t_array, u_con, 'b-', label='Original x')
# plt.title('control action')
# plt.xlabel('Time')
# plt.ylabel('Population')
# plt.legend()
# plt.grid(True)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In [ ]:
# Apply Improved DMDc
rank_p, rank_r = 10, 5
eigenvalues, modes, recons, bar_A, bar_B, bar_V, time_improved = Improved_DMDc_new(X_data, U_control, rank_p, rank_r)
eigenvalues_std, modes_std, recons_std, bar_A_std, bar_B_std, bar_V_std, time_standard = Standard_DMDC(X_data, U_control, rank_p, rank_r)

# Predict future states
X_data_test = np.vstack((x_solved[N_train], y_solved[N_train]))
z_k = bar_V.T @ X_data_test  # Reduced initial states
future_steps = N_t - N_train
Y_pred = np.zeros((2, future_steps))

z_current = z_k[:,0].reshape(-1, 1)  # Initial reduced state

for i in range(future_steps):
    u_current = u_con[N_train + i].reshape(1, -1)
    z_next = bar_A @ z_current + bar_B @ u_current
    Y_pred[:, i] = (bar_V @ z_next).reshape(2,)
    z_current = z_next

In [ ]:
print("the improved time", time_improved)
print("the standard time", time_standard)

In [ ]:
# compute the error between the original and reconstructed data
def compute_error(original, reconstructed):
    return np.linalg.norm(original - reconstructed) / np.linalg.norm(original)
error_x = compute_error(X_data[0, :-1], recons[0, :])
error_y = compute_error(X_data[1, :-1], recons[1, :])
print(f"Relative Error in x: {error_x:.4f}, Error in y: {error_y:.4f}")

# compute the root mean square error (RMSE) between the original and reconstructed data
def compute_rmse(original, reconstructed):
    return np.sqrt(np.mean((original - reconstructed) ** 2))
rmse_x = compute_rmse(X_data[0, :-1], recons[0, :])
rmse_y = compute_rmse(X_data[1, :-1], recons[1, :])
print(f"RMSE in x: {rmse_x:.4f}, RMSE in y: {rmse_y:.4f}")

In [ ]:
print(  "Eigenvalues of the reduced system matrix A:", eigenvalues)
print("DMD Modes shape: \n", modes)

In [ ]:
from pydmd import DMD, DMDc
from pydmd.plotter import plot_eigs
#Compute DMDc using the corresponding classes in PyDMD package

dmdc = DMDc(svd_rank=-1)
dmdc.fit(X_data, U_control)
# plot_eigs(dmdc)
print("DMDc Eigenvalues:", dmdc.eigs)
print("DMDc Modes shape: \n", dmdc.modes)
plot_eigs(dmdc)

In [ ]:
# Create a figure with subplots
plt.figure(figsize=(15, 10))

# # Subplot 1: State Trajectories
# plt.subplot(2, 2, 1)
plt.plot(t_array[:], x_solved[:], 'b-', alpha=0.5)
plt.plot(t_array[:], y_solved[:], 'g-', alpha=0.5)

# the prediction from pyDMDc

dmdc.dmd_time["dt"] = t_array[1] - t_array[0]
dmdc.dmd_time["t0"] = t_array[0]
dmdc.dmd_time["tend"] = t_array[-1]

# print("the shape of dynamics is", (dmdc.dynamics.shape[-1] - dmdc._lag))
# print("the shape of U", u_con.T.reshape(1,-1).real.shape)
# predicted_pydmdc = dmdc.reconstructed_data(u_con.T.reshape(1,-1).real)
# plt.plot(t_array[:], predicted_pydmdc[0, :-1], 'r.-', alpha=0.5, label='Reconstruction pyDMD x')
# plt.plot(t_array[:], predicted_pydmdc[1, :-1], 'm.-', alpha=0.5, label='Reconstruction pyDMD y')

plt.title('State Trajectories')
plt.xlabel('Time')
plt.ylabel('Population')
plt.legend()
plt.grid(True)

In [ ]:
# Create a figure with subplots
plt.figure(figsize=(15, 10))

# Subplot 1: State Trajectories
plt.subplot(2, 2, 1)
plt.plot(t_array[:N_train-1], X_data[0, :-1], 'b-', label='Original x')
plt.plot(t_array[:N_train-1], recons[0, :], 'r--', label='Reconstructed x')
plt.plot(t_array[:N_train-1], X_data[1, :-1], 'g-', label='Original y')
plt.plot(t_array[:N_train-1], recons[1, :], 'm--', label='Reconstructed y')
plt.plot(t_array[N_train:], x_solved[N_train:], 'b-', alpha=0.5)
plt.plot(t_array[N_train:], y_solved[N_train:], 'g-', alpha=0.5)
plt.plot(t_array[N_train:], Y_pred[0, :], 'r--', alpha=0.5)
plt.plot(t_array[N_train:], Y_pred[1, :], 'm--', alpha=0.5)

# the prediction from pyDMDc

dmdc.dmd_time["dt"] = t_array[1] - t_array[0]
dmdc.dmd_time["t0"] = t_array[0]
dmdc.dmd_time["tend"] = t_array[-1]
print("the shape of U", u_con.shape)
# predicted_pydmdc = dmdc.reconstructed_data(u_con.T[:-1].reshape(1,-1).real)
# plt.plot(t_array[N_train:], predicted_pydmdc[0, N_train:], 'r.-', alpha=0.5, label='Reconstruction pyDMD x')
# plt.plot(t_array[N_train:], predicted_pydmdc[1, N_train:], 'm.-', alpha=0.5, label='Reconstruction pyDMD y')

plt.title('State Trajectories')
plt.xlabel('Time')
plt.ylabel('Population')
plt.legend()
plt.grid(True)

# Subplot 2: Phase Portrait
plt.subplot(2, 2, 2)
plt.plot(X_data[0, :-1], X_data[1, :-1], 'b-', label='Original')
plt.plot(recons[0, :], recons[1, :], 'r--', label='Reconstructed')
plt.plot(x_solved[N_train-1:], y_solved[N_train-1:], 'b-', alpha=0.5)
plt.plot(Y_pred[0, :], Y_pred[1, :], 'r--', alpha=0.5)
plt.title('Phase Portrait')
plt.xlabel('x (Prey)')
plt.ylabel('y (Predator)')
plt.legend()
plt.grid(True)

# Subplot 3: Eigenvalues with Unit Circle
plt.subplot(2, 2, 3)
plt.scatter(eigenvalues.real, eigenvalues.imag, c='b', marker='o', label='Eigenvalues')
theta = np.linspace(0, 2 * np.pi, 400)
plt.plot(np.cos(theta), np.sin(theta), 'k--', label='Unit Circle')
plt.axhline(0, color='k', linestyle='--', linewidth=0.5)
plt.axvline(0, color='k', linestyle='--', linewidth=0.5)
plt.title('DMD Eigenvalues')
plt.xlabel('Real Part')
plt.ylabel('Imaginary Part')
plt.legend()
plt.axis('equal')
plt.grid(True)

# Subplot 4: DMD Modes (Eigenvectors)
plt.subplot(2, 2, 4)
origin = np.zeros((2, modes.shape[1]))

# Find axis limits to keep all vectors inside the figure
x_max = np.max(np.abs(modes[0, :].real)) * 1.2
y_max = np.max(np.abs(modes[1, :].real)) * 1.2

plt.quiver(origin[0], origin[1], modes[0, :].real, modes[1, :].real, angles='xy', scale_units='xy', scale=1)
for i in range(modes.shape[1]):
    plt.text(modes[0, i].real, modes[1, i].real, f'Mode {i+1}', fontsize=10)
plt.title('DMD Modes (Eigenvectors)')
plt.xlabel('x-component')
plt.ylabel('y-component')
plt.axvline(0, color='k', linestyle='--', linewidth=0.5)
plt.axhline(0, color='k', linestyle='--', linewidth=0.5)
plt.xlim(-x_max, x_max)
plt.ylim(-y_max, y_max)
plt.grid(True)

# # Adjust layout and save the figure
plt.tight_layout()
# plt.savefig('lotka_volterra_dmdc_analysis.png')
# plt.close()
plt.show()